In [12]:
!pip install tqdm

In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input
from tqdm import tqdm

In [14]:
dataset_name = "cats_vs_dogs"
dataset, info = tfds.load(dataset_name, as_supervised=True, with_info=True)
train_data, test_data = dataset['train'].take(20000), dataset['train'].skip(20000)

image_shape = (28, 28, 1)
latent_dim = 100

In [15]:
def preprocess_image(image, label):
    image = tf.image.resize(image, (28, 28))
    image = tf.image.rgb_to_grayscale(image)
    image = image / 255.0
    return image, label

In [16]:
X_train = train_data.map(preprocess_image)
X_train = X_train.batch(64)

In [17]:
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(7 * 7 * 256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(1, kernel_size=7, strides=1, padding='same', activation='tanh'))
    return model

In [18]:
def build_discriminator(image_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=image_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))  # Output: real or fake
    return model

In [19]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [20]:
# Compile models
discriminator = build_discriminator(image_shape)
discriminator.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
generator = build_generator(latent_dim)

gan = build_gan(generator, discriminator)
gan.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

In [21]:
def train_gan(epochs, batch_size, latent_dim, dataset):
    for epoch in range(epochs):
        with tqdm(total=len(dataset), desc=f"Epoch {epoch + 1}/{epochs}", ncols=100) as pbar:
            for real_images, _ in dataset:
                real_images = real_images[:batch_size]
                if real_images.shape[0] < batch_size:
                    continue

                noise = np.random.normal(0, 1, (batch_size, latent_dim))
                fake_images = generator.predict(noise)

                real_labels = np.ones((batch_size, 1))
                fake_labels = np.zeros((batch_size, 1))
                d_loss_real = discriminator.train_on_batch(real_images, real_labels)
                d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                noise = np.random.normal(0, 1, (batch_size, latent_dim))
                valid_labels = np.ones((batch_size, 1))
                g_loss = gan.train_on_batch(noise, valid_labels)

                pbar.set_postfix(D_loss=d_loss[0], G_loss=g_loss[0])
                pbar.update(1)

        print(f"Epoch {epoch + 1}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}")

In [ ]:
epochs = 4
batch_size = 64
train_gan(epochs, batch_size, latent_dim, X_train)

Epoch 1/30:   0%|                                                           | 0/313 [00:00<?, ?it/s]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


Epoch 1/30:   0%|                     | 1/313 [00:02<13:11,  2.54s/it, D_loss=0.7, G_loss=0.6978406]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


Epoch 1/30:   1%|▏                     | 2/313 [00:03<08:04,  1.56s/it, D_loss=0.7, G_loss=0.699096]

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


Epoch 4/30:  79%|██████████████▏   | 246/313 [05:14<01:26,  1.29s/it, D_loss=1.09, G_loss=1.0920725]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


Epoch 4/30:  79%|██████████████▏   | 247/313 [05:16<01:25,  1.29s/it, D_loss=1.09, G_loss=1.0921189]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


In [ ]:
def plot_generated_images(epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)
    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i] * 0.5 + 0.5)  # Rescale to [0, 1]
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f"generated_image_epoch_{epoch}.png")
    plt.show()